## Import library

In [1]:
import sys

MAIN_PATH = "/home/trduong/Data/interpretable_machine_learning/Source Code/my_work"


sys.path.insert(
    1,
    MAIN_PATH + '/lib'
)

sys.path.insert(
    1,
    MAIN_PATH + '/config'
)

import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import math
import timeit
import utils
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import KFold
import experiment_config as ecf

import dowhy.datasets
import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt

from econml.drlearner import ForestDRLearner, LinearDRLearner
from econml.metalearners import SLearner, XLearner, TLearner
from econml.ortho_forest import CausalTree, ContinuousTreatmentOrthoForest, DiscreteTreatmentOrthoForest
from econml.dml import ForestDMLCateEstimator, LinearDMLCateEstimator, SparseLinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.sklearn_extensions.linear_model import WeightedLasso, WeightedLassoCV

### Import sklearn
from scipy.stats import sem
import scipy.stats as st
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.linear_model import LassoCV, ElasticNetCV
from bartpy.sklearnmodel import SklearnModel


from zepid.causal.doublyrobust import TMLE
from cforest.forest import CausalForest
from imblearn.over_sampling import RandomOverSampler


/home/trduong/anaconda3/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/trduong/anaconda3/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


## Load data

In [19]:
df = pd.read_csv("/home/trduong/Data/data/training.csv")

In [20]:
df.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [21]:
df['treatment'] = np.where(df['Promotion'] == 'Yes', 1, 0)

## Build model

In [22]:
def score(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return (irr, nir)

In [23]:
col = list(df.columns)

treatment = 'treatment' # intervention, or treatment
outcome = 'purchase' # outcome of interest

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove("ID")
cov.remove("Promotion")
print(cov)

features = col[:]
features.remove(outcome)
features.remove("ID")
features.remove("Promotion")
print(features)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']
['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'treatment']


In [24]:
## Fit treatment
X = df[cov]
y = df[treatment]
ros = RandomOverSampler(random_state=42)
X_sampling, y_sampling = ros.fit_resample(X, y)
model_t = LogisticRegression()
model_t.fit(X_sampling, y_sampling)

## Fit outcome
X = df[features]
y = df[outcome]
ros = RandomOverSampler(random_state=42)
X_sampling, y_sampling = ros.fit_resample(X, y)
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 50)
model_y.fit(X_sampling, y_sampling)

GradientBoostingRegressor(n_estimators=50, random_state=0)

## Evaluation

In [42]:
def score(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return nir

def fitness_function(delta, solution_idx):
    propensity_score = (delta * df['p1']) / (delta * df['p1'] + df['p0'])
    df['Promotion'] = np.where(propensity_score >= 0.5, 'Yes', 'No')
    return score(df, 'Promotion')



In [43]:
df.shape

In [44]:
score(df, promo_pred_col = 'Promotion')

## Explore the changes in data

In [45]:
df['p1'] = model_t.predict_proba(df[cov].values)[:,:1]
df['p0'] = 1 - model_t.predict_proba(df[cov].values)[:,:1]

In [46]:
df.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7,treatment,p1,p0
0,1,No,0,2,30.443518,-1.165083,1,1,3,2,0,0.499249,0.500751
1,3,No,0,3,32.159350,-0.645617,2,3,2,2,0,0.500745,0.499255
2,4,No,0,2,30.431659,0.133583,1,1,4,2,0,0.493894,0.506106
3,5,No,0,0,26.588914,-0.212728,2,1,4,2,0,0.502509,0.497491
4,8,No,0,3,28.044332,-0.385883,1,1,2,2,1,0.493804,0.506196


In [47]:
# init_delta = np.ones(df.shape[0])
# init = init_delta*df['p1'].values
# init = [init]*1000

In [48]:
import pygad
import numpy

sols = []

function_inputs = np.random.rand(df.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 80 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0

def callback_generation(ga_instance):
    global last_fitness
    global sols
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    sols.append(solution)

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Running model
Generation = 1
Fitness    = -10112.25
Change     = -10112.25


Generation = 2
Fitness    = -10112.25
Change     = 0.0


Generation = 3
Fitness    = -10112.25
Change     = 0.0


Generation = 4
Fitness    = -10112.25
Change     = 0.0


Generation = 5
Fitness    = -10112.25
Change     = 0.0


Generation = 6
Fitness    = -10112.25
Change     = 0.0


Generation = 7
Fitness    = -10112.25
Change     = 0.0


Generation = 8
Fitness    = -10112.25
Change     = 0.0


Generation = 9
Fitness    = -10112.25
Change     = 0.0


Generation = 10
Fitness    = -10112.25
Change     = 0.0


Generation = 11
Fitness    = -10112.25
Change     = 0.0


Generation = 12
Fitness    = -10112.25
Change     = 0.0


Generation = 13
Fitness    = -10112.25
Change     = 0.0


Generation = 14
Fitness    = -10112.25
Change     = 0.0


Generation = 15
Fitness    = -10112.25
Change     = 0.0


Generation = 16
Fitness    = -10112.25
Change     = 0.0


Generation = 17
Fitness    = -10112.25
Change     = 0.0


Gen

KeyboardInterrupt: 

In [39]:
sols[0]

array([ 0.56083841, -0.59056712, -0.64146736, ...,  0.27929713,
        0.43813204,  0.63347145])

In [40]:
sols[-1]

array([ 0.56083841, -0.59056712, -0.64146736, ...,  0.27929713,
        0.43813204,  0.63347145])

In [41]:
sols[1]

array([ 0.56083841, -0.59056712, -0.64146736, ...,  0.27929713,
        0.43813204,  0.63347145])